In [ ]:
!pip install torch torchvision torchaudio torchtext
!pip install pytorch-forecasting
!pip install mlxtend
!pip install torchmetrics

In [ ]:
import torch.utils.data as data

# DeepGA is an algorithm for evolve a convolutional neural network.


In [2]:
from DeepGA.Operators import *
from DeepGA.EncodingClass import Encoding
from DeepGA.Decoding import *
from DeepGA.DataReader import *
from DeepGA.DistributedTraining import *
from DeepGA.DeepGA import *

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
 %cd /content/drive/MyDrive/DeepGA_kvasir

# Data reader

In [4]:
def loading_data():
# preprocessing
    print(f"MedMNIST v{medmnist.__version__} @ {medmnist.HOMEPAGE}")

    #flag = "pathmnist"
    info_pathmnist = INFO['pathmnist']
    DataClass_pathmnist = getattr(medmnist, info_pathmnist['python_class'])
    data_transform = transforms.Compose([
       transforms.ToTensor(),  
    ])
    train_data = DataClass_pathmnist(split='train', transform=data_transform, download=True)
    val_data = DataClass_pathmnist(split='val', transform=data_transform, download=True)
    test_data = DataClass_pathmnist(split='test', transform=data_transform, download=True)
    
    BATCH_SIZE = 128
    train_dl = data.DataLoader(train_data, batch_size= BATCH_SIZE, shuffle = True)
    val_dl = data.DataLoader(val_data, batch_size= BATCH_SIZE, shuffle = False)
    test_dl = data.DataLoader(test_data, batch_size= BATCH_SIZE, shuffle = False)
    
    data_flag = 'pathmnist'
    info = INFO[data_flag]
    task = info['task']
    n_channels = info['n_channels']
    n_classes = len(info['label'])

    return train_dl, val_dl, test_dl, n_channels, n_classes, 28

### Call DeepGA for evolve CNN´s

In [ ]:
'''Defining DeepGA hyperparameters'''
#Convolutional layers
FSIZES = [3, 5, 7, 9] # Odd Sizes Are Preferred
NFILTERS = [8, 16, 32, 64, 128, 256]

#Pooling layers
PSIZES = [2,3] #[2,3,4,5]
PTYPE = ['max', 'avg']

#Fully connected layers
NEURONS = [16, 32, 64, 128, 256] # for big layers

#Defining learning rate
lr = 1e-4

#Maximun and minimum numbers of layers to initialize networks
min_conv = 2 # 30
max_conv = 30 # 60
min_full = 1
max_full = 3 # 10
max_params = 5e6
train_epochs = 5 # Epochs to train the best individual found by the GA

'''Genetic Algorithm Parameters'''
cr = 0.7   # Crossover rate
mr = 0.5   # Mutation rate
N = 20      # Population size 20 Se mantuvo en 20
T = 30      # Number of generations
t_size = 5 # tournament size 5
w = 0.1    # penalization weight   0.3
#chck_dir = '/content/drive/MyDrive/checkpoint/'  # Root folder for dumping/loading pickle
chck_dir = 'point/'  # Root folder for dumping/loading pickle
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
train_dl, val_dl, test_dl, n_channels, n_classes, out_size = loading_data() #Loading data

loss_func = nn.CrossEntropyLoss()
execution_ID = 707 #Execution ID for checkpoint, change it for a new execution
results, pop, bestind  = deepGA(execution_ID, True, train_epochs = train_epochs, train_dl=train_dl, val_dl=val_dl,  lr=lr,
                       min_conv=min_conv, max_conv=max_conv, min_full=min_full, max_full=max_full, max_params=max_params,
                       cr=cr, mr=mr, N=N, T=T, t_size=t_size, w=w, device=device, chck_dir=chck_dir,
                       n_channels =  n_channels , n_classes=n_classes, out_size = out_size, loss_func=loss_func)

# training to more epochs

finalEpochs = 100
CNNModel = final_evaluation(execution_ID, bestind, train_dl, val_dl, lr, max_params, w, device, finalEpochs, loss_func, chck_dir, n_channels =  n_channels , n_classes=n_classes, out_size = out_size)

MedMNIST v3.0.2 @ https://github.com/MedMNIST/MedMNIST/
Using downloaded and verified file: C:\Users\jose_\.medmnist\pathmnist.npz
Using downloaded and verified file: C:\Users\jose_\.medmnist\pathmnist.npz
Using downloaded and verified file: C:\Users\jose_\.medmnist\pathmnist.npz
Using downloaded and verified file: C:\Users\jose_\.medmnist\pathmnist.npz
Using downloaded and verified file: C:\Users\jose_\.medmnist\pathmnist.npz
Re-Initialize population
The maximum number of generations has been reached. Please run a new execution.
--------------------------------------------
    Accuracy   Fitness  No. Params   MeanFit   MeanAcc    MeanPar
0   0.917033  0.914662      533393  0.850344  0.865204  1416999.4
1   0.917033  0.914662      533393  0.888973  0.895127   832081.8
2   0.912435  0.918521      133529  0.904107  0.905838   557350.2
3   0.919932  0.919114      441217  0.911085  0.911650   470007.0
4   0.919932  0.919114      441217  0.913154  0.913500   449759.0
5   0.919932  0.919114 